# Read Dimension

同一のディメンジョンのデータを収集する


In [133]:
import os
from pathlib import Path
import edinet

# サントリーホールディングス株式会社/H31.03.26 12:50
DOC_ID = "S100FGSC"

# Data Folder
DATA_ROOT = Path.cwd().joinpath("data")

## Download XBRL

In [134]:
from edinet.xbrl_file import XBRLDir


# Download and load document
xbrl_path = edinet.api.document.get_xbrl(
    DOC_ID, save_dir=DATA_ROOT.joinpath("raw"),
    expand_level="dir")

xbrl_dir = XBRLDir(xbrl_path)

c:\Users\ico\Documents\works\xbrl_read_tutorial\venv\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


## Find tags with same dimension and member



In [145]:
from lxml import etree
import pandas as pd


# Namespace is defined at
# http://www.xbrl.org/Specification/XBRL-2.1/REC-2003-12-31/XBRL-2.1-REC-2003-12-31+corrected-errata-2013-02-20.html#_1.6
# https://www.xbrl.org/specification/dimensions/rec-2012-01-25/dimensions-rec-2006-09-18+corrected-errata-2012-01-25-clean.html

non_consolidated_dimension = "jppfs_cor:ConsolidatedOrNonConsolidatedAxis"
equity_axis = "jppfs_cor:ComponentsOfEquityAxis"
equity_member = "jppfs_cor:ShareholdersEquityMember"
non_consolidated_dimension
root = etree.parse(xbrl_dir.xbrl.path)
query = f"""
xbrli:context[
    xbrli:scenario[
        xbrldi:explicitMember[@dimension='{equity_axis}']
    ]
]
"""

equities_on_equity_axis = root.xpath(query, namespaces={
    "xbrli": "http://www.xbrl.org/2003/instance",
    "xbrldi": "http://xbrl.org/2006/xbrldi"
})


data = []
for e in equities_on_equity_axis:
    _id = e.attrib["id"]
    member = e.find(f".//xbrldi:explicitMember[@dimension='{equity_axis}']",
                    namespaces=namespaces).text

    period = ""
    period_begin = ""
    period_element = e.find(".//xbrli:period/xbrli:instant", namespaces=namespaces)
    if period_element is not None:
        period = period_element.text
    else:
        period_element = e.find(".//xbrli:period", namespaces=namespaces)
        for p in period_element.getchildren():
            if p.tag.endswith("startDate"):
                period_begin = p.text
            else:
                period = p.text

    if _id and period:
        tag = xbrl_dir.xbrl.find(None, contextRef=_id)
        data.append({
            "member": member,
            "period": period,
            "period_begin": period_begin,
            "element": tag.element.name,
            "value": tag.text
        })

equities = pd.DataFrame(data)

In [155]:
equities[(equities["period"] == "2018-12-31") & (equities["period_begin"] == "")]

,member,period,period_begin,element,value
0,jppfs_cor:ShareholdersEquityMember,2018-12-31,,NetAssets,740191000000
1,jppfs_cor:CapitalStockMember,2018-12-31,,NetAssets,70000000000
2,jppfs_cor:CapitalSurplusMember,2018-12-31,,NetAssets,286367000000
24,jppfs_cor:RetainedEarningsBroughtForwardMember,2018-12-31,,NetAssets,70243000000
25,jppfs_cor:TreasuryStockMember,2018-12-31,,NetAssets,-938000000
26,jppfs_cor:ValuationAndTranslationAdjustmentsMe...,2018-12-31,,NetAssets,2999000000
27,jppfs_cor:ValuationDifferenceOnAvailableForSal...,2018-12-31,,NetAssets,6250000000
28,jppfs_cor:DeferredGainsOrLossesOnHedgesMember,2018-12-31,,NetAssets,-3251000000
55,jppfs_cor:LegalCapitalSurplusMember,2018-12-31,,NetAssets,17500000000
56,jppfs_cor:OtherCapitalSurplusMember,2018-12-31,,NetAssets,268867000000
